In [11]:
import psycopg2


conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="obama")



In [12]:
cur = conn.cursor()

# Drop all tables if they exist
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
tables = cur.fetchall()
for table in tables:
    cur.execute(f"DROP TABLE IF EXISTS {table[0]} CASCADE;")
conn.commit()

with open('dump.sql', 'r') as sql_file:
    sql_script = sql_file.read()

# Execute the SQL script to create tables
cur.execute(sql_script)

# Commit the changes to apply the table creations
conn.commit()

In [13]:
from DataFaker import DataFaker
import random

cur = conn.cursor()

faker = DataFaker()

for i in faker.si_sites_data:
    sql = """INSERT INTO si_sites(si_name) VALUES(%s)"""
    cur.execute(sql, i)
conn.commit()

for i in faker.f_formats_data:
    sql = """INSERT INTO f_formats(f_name)
             VALUES(%s);"""
    cur.execute(sql, i)
conn.commit()

for i in faker.s_sessions_data:
    sql = """INSERT INTO s_sessions(s_date, s_length)
             VALUES(%s, %s);"""
    cur.execute(sql, i)
conn.commit()

for i in faker.t_tournaments_data:
    sql = """INSERT INTO t_tournaments(t_name, t_buyIn, t_guarantee, t_si_site)
             VALUES(%s, %s, %s, %s);"""
    cur.execute(sql, i)
conn.commit()

for tournament, formats in faker.tournament_formats_mapping.items():
    for f_id in formats:
        sql = """INSERT INTO hf_hasFormat(hf_f_id, hf_t_id)
                 VALUES(%s, %s);"""
        cur.execute(sql, (f_id, tournament))
conn.commit()

for session, tournaments in faker.session_tournaments_mapping.items():
    for t_id in tournaments:
        sql = """INSERT INTO ht_hasTournament(ht_s_id, ht_t_id, ht_winnings, ht_position)
                 VALUES(%s, %s, %s, %s);"""
        cur.execute(sql, (session, t_id, random.randint(0, 10000), random.randint(1, 500)))
conn.commit()

for i in faker.tr_transactions_data:
    sql = """INSERT INTO tr_transactions(tr_si_id, tr_amount, tr_date)
             VALUES(%s, %s, %s);"""
    cur.execute(sql, i)
conn.commit()

print("Done")

Done
